# Converting from grid-based to municipality-based 
#### GridGlobal++

In [1]:
%load_ext jupyter_black

In [30]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import shap
import imblearn
import statsmodels.api as sm
import statistics
import warnings


from math import sqrt
from collections import defaultdict
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.preprocessing import RobustScaler
from sklearn import metrics
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import mean_squared_error
from xgboost.sklearn import XGBRegressor
from sklearn.dummy import DummyRegressor
from xgboost import XGBClassifier
from sty import fg, rs

from sklearn.metrics import confusion_matrix
from matplotlib import cm
from collections import Counter
from imblearn.under_sampling import RandomUnderSampler

from utils import get_training_dataset, weight_file

In [31]:
# Hide all warnings
warnings.filterwarnings("ignore")

In [32]:
# Import the created dataset to a df
df = get_training_dataset()
df

,typhoon_name,typhoon_year,grid_point_id,wind_speed,track_distance,rainfall_max_6h,rainfall_max_24h,total_houses,rwi,strong_roof_strong_wall,...,std_tri,mean_elev,coast_length,with_coast,urban,rural,water,total_pop,percent_houses_damaged,percent_houses_damaged_5years
0,DURIAN,2006,101,0.0,303.180555,0.122917,0.085417,31.000000,NaN,22.580645,...,2.699781,5.762712,3445.709753,1,0.00,0.000000,1.000000,0.000000,0.0,0.000000
1,DURIAN,2006,4475,0.0,638.027502,0.091667,0.027083,3.301020,-0.527000,2.639401,...,4.585088,12.799127,8602.645832,1,0.00,0.000000,1.000000,0.000000,0.0,0.000000
2,DURIAN,2006,4639,0.0,603.631997,0.535417,0.146354,12.103741,-0.283000,2.639401,...,1.527495,8.833333,5084.012925,1,0.00,0.010000,0.990000,197.339034,0.0,0.000000
3,DURIAN,2006,4640,0.0,614.675270,0.356250,0.101562,645.899660,-0.358889,2.639401,...,11.677657,17.530431,55607.865950,1,0.00,0.310000,0.690000,4970.477311,0.0,0.000000
4,DURIAN,2006,4641,0.0,625.720905,0.202083,0.057812,1071.731293,-0.462800,2.639401,...,17.074011,31.931338,35529.342507,1,0.00,0.770000,0.230000,12408.594656,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
141253,MOLAVE,2020,20677,0.0,644.575831,2.543750,0.778646,4449.357133,0.508167,44.762048,...,18.012771,36.304688,21559.003490,1,0.08,0.080000,0.840000,17619.701390,0.0,0.000000
141254,MOLAVE,2020,20678,0.0,655.685233,2.558333,0.861458,1521.435795,-0.174100,44.762048,...,13.163042,65.687266,12591.742022,1,0.00,0.420000,0.580000,5623.069564,0.0,0.000000
141255,MOLAVE,2020,20679,0.0,666.794635,2.975000,0.949479,930.647069,-0.244286,25.078318,...,10.901755,37.414996,19740.596834,1,0.00,0.109091,0.890909,5912.671746,0.0,0.015207
141256,MOLAVE,2020,20680,0.0,677.904037,2.889583,1.083333,1800.666044,0.038000,16.796996,...,17.917650,105.812452,26363.303778,1,0.03,0.250000,0.720000,11254.164413,0.0,0.020806


In [33]:
for i in range(len(df)):
    df.at[i, "typhoon_year"] = str(df.loc[i]["typhoon_year"])

In [34]:
df["typhoon_name"] = df["typhoon_name"] + df["typhoon_year"]

In [36]:
# Set any values >100% to 100%,
for i in range(len(df)):
    if df.loc[i, "percent_houses_damaged"] > 100:
        df.at[i, "percent_houses_damaged"] = float(100)

In [37]:
# Fill NaNs with average estimated value of 'rwi'
df["rwi"].fillna(df["rwi"].mean(), inplace=True)

In [38]:
# Read the new weight CSV file and import to a df
df_weight = weight_file("/ggl_grid_to_mun_weights.csv")
df_weight.head()

,ADM3_PCODE,id_x,Centroid,numbuildings_x,id,numbuildings,weight
0,PH012801000,11049.0,120.9E_18.5N,1052,11049,1794,0.586399
1,PH012810000,11049.0,120.9E_18.5N,0,11049,1794,0.000000
2,PH012815000,11049.0,120.9E_18.5N,742,11049,1794,0.413601
3,PH012801000,11050.0,120.9E_18.4N,193,11050,196,0.984694
4,PH012810000,11050.0,120.9E_18.4N,0,11050,196,0.000000


In [39]:
# Change name of column ['id'] to ['grid_point_id'] the same name as in input df
df_weight.rename(columns={"id": "grid_point_id"}, inplace=True)
df_weight.head()

,ADM3_PCODE,id_x,Centroid,numbuildings_x,grid_point_id,numbuildings,weight
0,PH012801000,11049.0,120.9E_18.5N,1052,11049,1794,0.586399
1,PH012810000,11049.0,120.9E_18.5N,0,11049,1794,0.000000
2,PH012815000,11049.0,120.9E_18.5N,742,11049,1794,0.413601
3,PH012801000,11050.0,120.9E_18.4N,193,11050,196,0.984694
4,PH012810000,11050.0,120.9E_18.4N,0,11050,196,0.000000


### Following Steps are to convert grid_based model into Municipality based one

In [40]:
# Remove zeros from wind_speed
df = df[(df[["wind_speed"]] != 0).any(axis=1)]
df_data = df.drop(columns=["grid_point_id", "typhoon_year"])

In [41]:
display(df.head())
display(df_data.head())

,typhoon_name,typhoon_year,grid_point_id,wind_speed,track_distance,rainfall_max_6h,rainfall_max_24h,total_houses,rwi,strong_roof_strong_wall,...,std_tri,mean_elev,coast_length,with_coast,urban,rural,water,total_pop,percent_houses_damaged,percent_houses_damaged_5years
138,DURIAN2006,2006,8284,12.460039,275.018491,0.670833,0.313021,0.479848,-0.213039,31.336503,...,34.629550,42.218750,5303.659490,1,0.00,0.00,1.00,0.000000,0.0,0.0
139,DURIAN2006,2006,8286,11.428974,297.027578,0.929167,0.343229,55.649739,0.206000,23.447758,...,25.475388,72.283154,61015.543599,1,0.00,0.14,0.86,276.871504,0.0,0.0
148,DURIAN2006,2006,8450,13.077471,262.598363,0.716667,0.424479,8.157414,-0.636000,31.336503,...,54.353996,102.215198,66707.438070,1,0.00,0.11,0.89,448.539453,0.0,0.0
149,DURIAN2006,2006,8451,12.511864,273.639330,0.568750,0.336979,88.292015,-0.227500,31.336503,...,31.814048,58.988877,53841.050168,1,0.00,0.12,0.88,2101.708435,0.0,0.0
150,DURIAN2006,2006,8452,11.977511,284.680297,0.589583,0.290625,962.766739,-0.299667,23.546053,...,25.976413,111.386527,87378.257957,1,0.07,0.46,0.47,11632.726327,0.0,0.0


,typhoon_name,wind_speed,track_distance,rainfall_max_6h,rainfall_max_24h,total_houses,rwi,strong_roof_strong_wall,strong_roof_light_wall,strong_roof_salvage_wall,...,std_tri,mean_elev,coast_length,with_coast,urban,rural,water,total_pop,percent_houses_damaged,percent_houses_damaged_5years
138,DURIAN2006,12.460039,275.018491,0.670833,0.313021,0.479848,-0.213039,31.336503,29.117802,0.042261,...,34.629550,42.218750,5303.659490,1,0.00,0.00,1.00,0.000000,0.0,0.0
139,DURIAN2006,11.428974,297.027578,0.929167,0.343229,55.649739,0.206000,23.447758,23.591571,0.037516,...,25.475388,72.283154,61015.543599,1,0.00,0.14,0.86,276.871504,0.0,0.0
148,DURIAN2006,13.077471,262.598363,0.716667,0.424479,8.157414,-0.636000,31.336503,29.117802,0.042261,...,54.353996,102.215198,66707.438070,1,0.00,0.11,0.89,448.539453,0.0,0.0
149,DURIAN2006,12.511864,273.639330,0.568750,0.336979,88.292015,-0.227500,31.336503,29.117802,0.042261,...,31.814048,58.988877,53841.050168,1,0.00,0.12,0.88,2101.708435,0.0,0.0
150,DURIAN2006,11.977511,284.680297,0.589583,0.290625,962.766739,-0.299667,23.546053,23.660429,0.037576,...,25.976413,111.386527,87378.257957,1,0.07,0.46,0.47,11632.726327,0.0,0.0


In [42]:
# Read municipality dataset which already merged with y_norm converted ground truth
df_mun_merged = pd.read_csv("data/df_merged_2.csv")

# Remove the duplicated rows
df_mun_merged.drop_duplicates(keep="first", inplace=True)
df_mun_merged = df_mun_merged.reset_index(drop=True)

# Make the name of typhoons to uppercase
df_mun_merged["typhoon"] = df_mun_merged["typhoon"].str.upper()

# Rename y_norm column
df_mun_merged = df_mun_merged.rename(columns={"y_norm": "y_norm_mun"})

df_mun_merged

,Mun_Code,typhoon,HAZ_rainfall_Total,HAZ_rainfall_max_6h,HAZ_rainfall_max_24h,HAZ_v_max,HAZ_dis_track_min,GEN_landslide_per,GEN_stormsurge_per,GEN_Bu_p_inSSA,...,VUL_LightRoof_LightWall,VUL_LightRoof_SalvageWall,VUL_SalvagedRoof_StrongWall,VUL_SalvagedRoof_LightWall,VUL_SalvagedRoof_SalvageWall,VUL_vulnerable_groups,VUL_pantawid_pamilya_beneficiary,DAM_perc_dmg,HAZ_v_max_3,y_norm_mun
0,PH175101000,DURIAN2006,185.828571,14.716071,7.381696,55.032241,2.478142,2.64,6.18,6.18,...,41.892832,1.002088,0.000000,0.027836,0.083507,2.951511,46.931106,3.632568,166667.757548,3.34975
1,PH083701000,DURIAN2006,8.818750,0.455208,0.255319,8.728380,288.358553,0.06,0.00,0.00,...,13.645253,0.549120,0.030089,0.090266,0.112833,3.338873,25.989168,0.000000,664.968323,0.00000
2,PH015501000,DURIAN2006,24.175000,2.408333,0.957639,10.945624,274.953818,1.52,1.28,1.28,...,15.592295,0.075838,0.000000,0.015168,0.075838,2.131755,32.185651,0.000000,1311.358762,0.00000
3,PH015502000,DURIAN2006,14.930000,1.650000,0.586250,12.108701,252.828578,0.00,0.00,0.00,...,7.100454,0.023280,0.011640,0.000000,0.128041,1.589369,29.612385,0.000000,1775.385328,0.00000
4,PH175302000,DURIAN2006,13.550000,1.054167,0.528125,10.660943,258.194381,5.52,0.36,0.36,...,30.354796,0.000000,0.000000,0.032852,0.000000,1.387007,35.052562,0.000000,1211.676901,0.00000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7855,PH084823000,NOUL2015,9.700000,0.408333,0.216146,8.136932,277.107823,1.80,6.25,6.25,...,32.492212,0.311526,0.031153,0.155763,0.031153,2.827833,31.308411,0.000000,538.743551,0.00000
7856,PH015547000,NOUL2015,17.587500,1.414583,0.386458,9.818999,305.789817,0.00,0.00,0.00,...,4.703833,0.027875,0.000000,0.034843,0.097561,1.073268,12.766551,0.000000,946.676507,0.00000
7857,PH025014000,NOUL2015,11.487500,0.614583,0.230319,15.791907,210.313249,0.06,0.09,0.09,...,3.063753,0.022528,0.000000,0.067583,0.022528,1.140109,9.348952,0.000000,3938.254316,0.00000
7858,PH140127000,NOUL2015,11.600000,1.400000,0.412766,13.867145,218.189328,0.00,0.00,0.00,...,3.119093,0.000000,0.000000,0.000000,0.000000,2.837537,21.928166,0.000000,2666.620370,0.00000


In [43]:
# Specify features
features = [
    "wind_speed",
    "track_distance",
    "total_houses",
    "rainfall_max_6h",
    "rainfall_max_24h",
    "rwi",
    # "strong_roof_strong_wall",
    # "strong_roof_light_wall",
    # "strong_roof_salvage_wall",
    # "light_roof_strong_wall",
    # "light_roof_light_wall",
    # "light_roof_salvage_wall",
    # "salvaged_roof_strong_wall",
    # "salvaged_roof_light_wall",
    # "salvaged_roof_salvage_wall",
    "mean_slope",
    "std_slope",
    "mean_tri",
    "std_tri",
    "mean_elev",
    "coast_length",
    "with_coast",
    "urban",
    "rural",
    "water",
    "total_pop",
    "percent_houses_damaged_5years",
]

# Split X and y from dataframe features
X = df_data[features]
display(X.columns)
y = df_data["percent_houses_damaged"]

scaler = preprocessing.StandardScaler().fit(X)
X_scaled = scaler.transform(X)

Index(['wind_speed', 'track_distance', 'total_houses', 'rainfall_max_6h',
       'rainfall_max_24h', 'rwi', 'mean_slope', 'std_slope', 'mean_tri',
       'std_tri', 'mean_elev', 'coast_length', 'with_coast', 'urban', 'rural',
       'water', 'total_pop', 'percent_houses_damaged_5years'],
      dtype='object')

In [44]:
# Define bins
bins2 = [0, 0.00009, 1, 10, 50, 101]
samples_per_bin2, binsP2 = np.histogram(df_data["percent_houses_damaged"], bins=bins2)

In [45]:
# Define range of for loop
num_exp = 20

# Define number of bins
num_bins = len(bins2)

In [54]:
M1_RMSE_lst = defaultdict(list)
Combined_RMSE_lst = defaultdict(list)

In [55]:
# Define empty list to save RMSE in combined model
test_RMSE_lst = np.zeros(num_exp)
test_RMSE_bin = np.zeros((num_exp, num_bins))

# Define empty list to save RMSE in model1
test_RMSE_lst_M1 = np.zeros(num_exp)
test_RMSE_bin_M1 = np.zeros((num_exp, num_bins))

In [56]:
# Defin two lists to save RMSE and Average Error

RMSE = defaultdict(list)
AVE = defaultdict(list)

In [57]:
for run_ix in range(num_exp):

    bin_index2 = np.digitize(df_data["percent_houses_damaged"], bins=binsP2)
    y_input_strat = bin_index2

    X = df_data[features]
    y = df_data["percent_houses_damaged"]

    # Define train and test data
    X_train, X_test, y_train, y_test = train_test_split(
        X, df_data["percent_houses_damaged"], test_size=0.2, stratify=y_input_strat
    )

    # XGBoost Reduced Overfitting
    xgb = XGBRegressor(
        base_score=0.5,
        booster="gbtree",
        colsample_bylevel=0.8,
        colsample_bynode=0.8,
        colsample_bytree=0.8,
        gamma=3,
        eta=0.01,
        importance_type="gain",
        learning_rate=0.1,
        max_delta_step=0,
        max_depth=4,
        min_child_weight=1,
        missing=1,
        n_estimators=100,
        early_stopping_rounds=10,
        n_jobs=1,
        nthread=None,
        objective="reg:squarederror",
        reg_alpha=0,
        reg_lambda=1,
        scale_pos_weight=1,
        seed=None,
        silent=None,
        subsample=0.8,
        verbosity=0,
        eval_metric=["rmse", "logloss"],
        random_state=0,
    )

    eval_set = [(X_train, y_train)]
    xgb_model = xgb.fit(X_train, y_train, eval_set=eval_set, verbose=False)

    # Make prediction on train and test data
    y_pred_train = xgb.predict(X_train)
    y_pred = xgb.predict(X_test)

    # Calculate RMSE in total
    mse_train_idx = mean_squared_error(y_train, y_pred_train)
    rmse_train = np.sqrt(mse_train_idx)

    mse_idx = mean_squared_error(y_test, y_pred)
    rmseM1 = np.sqrt(mse_idx)

    # Add total RMSE of Model1 to the list
    test_RMSE_lst_M1[run_ix] = rmseM1

    # Calculate RMSE per bins
    bin_index_test = np.digitize(y_test, bins=binsP2)
    bin_index_train = np.digitize(y_train, bins=binsP2)

    RSME_test_model1 = np.zeros(num_bins - 1)

    for bin_num in range(1, num_bins):

        # Estimation of RMSE for train data
        mse_train_idx = mean_squared_error(
            y_train[bin_index_train == bin_num],
            y_pred_train[bin_index_train == bin_num],
        )
        rmse_train = np.sqrt(mse_train_idx)

        # Estimation of RMSE for test data
        mse_idx = mean_squared_error(
            y_test[bin_index_test == bin_num], y_pred[bin_index_test == bin_num]
        )

        RSME_test_model1 = np.sqrt(mse_idx)

        # Add RMSE of Model1 to the list of each bin
        test_RMSE_bin_M1[run_ix, bin_num] = RSME_test_model1
        M1_RMSE_lst[bin_num].append(RSME_test_model1)

    ## Second step is to train XGBoost Binary model for same train data

    # Define a threshold to separate target into damaged and not_damaged
    thres = 10.0
    y_test_bool = y_test >= thres
    y_train_bool = y_train >= thres
    y_test_bin = (y_test_bool) * 1
    y_train_bin = (y_train_bool) * 1

    sum(y_train_bin)

    # Define undersampling strategy
    under = RandomUnderSampler(sampling_strategy=0.1)
    # Fit and apply the transform
    X_train_us, y_train_us = under.fit_resample(X_train, y_train_bin)

    # Use XGBClassifier as a Machine Learning model to fit the data
    xgb_model = XGBClassifier(eval_metric=["error", "logloss"])

    eval_set = [(X_train, y_train_bin)]
    xgb_model.fit(
        X_train_us,
        y_train_us,
        eval_set=eval_set,
        verbose=False,
    )

    # Make prediction on test data and print Confusion Matrix
    y_pred_test = xgb_model.predict(X_test)
    cm = confusion_matrix(y_test_bin, y_pred_test)

    # Make prediction on train data and print Confusion Matrix
    y_pred_train = xgb_model.predict(X_train)
    cm = confusion_matrix(y_train_bin, y_pred_train)

    reduced_df = X_train.copy()

    reduced_df["percent_houses_damaged"] = y_train.values
    reduced_df["predicted_value"] = y_pred_train

    fliterd_df = reduced_df[reduced_df.predicted_value == 1]

    ### Third step is to train XGBoost regression model for this reduced train data (including damg>10.0%)
    bin_index2 = np.digitize(fliterd_df["percent_houses_damaged"], bins=binsP2)
    y_input_strat = bin_index2

    # Split X and y from dataframe features
    X_r = fliterd_df[features]
    y_r = fliterd_df["percent_houses_damaged"]

    # XGBoost Reduced Overfitting
    xgbR = XGBRegressor(
        base_score=0.5,
        booster="gbtree",
        colsample_bylevel=0.8,
        colsample_bynode=0.8,
        colsample_bytree=0.8,
        gamma=3,
        eta=0.01,
        importance_type="gain",
        learning_rate=0.1,
        max_delta_step=0,
        max_depth=4,
        min_child_weight=1,
        missing=1,
        n_estimators=100,
        early_stopping_rounds=10,
        n_jobs=1,
        nthread=None,
        objective="reg:squarederror",
        reg_alpha=0,
        reg_lambda=1,
        scale_pos_weight=1,
        seed=None,
        silent=None,
        subsample=0.8,
        verbosity=0,
        eval_metric=["rmse", "logloss"],
        random_state=0,
    )

    eval_set = [(X_r, y_r)]
    xgbR_model = xgbR.fit(X_r, y_r, eval_set=eval_set, verbose=False)

    # Make prediction on train and global test data
    y_pred_r = xgbR.predict(X_r)
    y_pred_test_total = xgbR.predict(X_test)

    # Calculate RMSE in total
    mse_train_idxR = mean_squared_error(y_r, y_pred_r)
    rmse_trainR = np.sqrt(mse_train_idxR)

    mse_idxR = mean_squared_error(y_test, y_pred_test_total)
    rmseR = np.sqrt(mse_idxR)

    # Calculate RMSE per bins
    bin_index_r = np.digitize(y_r, bins=binsP2)

    RSME_test_model1R = np.zeros(num_bins - 1)

    for bin_num in range(1, num_bins):

        # Estimation of RMSE for train data
        mse_train_idxR = mean_squared_error(
            y_r[bin_index_r == bin_num], y_pred_r[bin_index_r == bin_num]
        )
        rmse_trainR = np.sqrt(mse_train_idxR)

        # Estimation of RMSE for test data
        mse_idxR = mean_squared_error(
            y_test[bin_index_test == bin_num],
            y_pred_test_total[bin_index_test == bin_num],
        )
        RSME_test_model1R[bin_num - 1] = np.sqrt(mse_idxR)

    #### Last step is to add model combination (model M1 with model MR)

    # Check the result of classifier for test set
    reduced_test_df = X_test.copy()

    # joined X_test with countinous target and binary predicted values
    reduced_test_df["percent_houses_damaged"] = y_test.values
    reduced_test_df["predicted_value"] = y_pred_test

    # damaged prediction
    fliterd_test_df1 = reduced_test_df[reduced_test_df.predicted_value == 1]

    # not damaged prediction
    fliterd_test_df0 = reduced_test_df[reduced_test_df.predicted_value == 0]

    # Use X0 and X1 for the M1 and MR models' predictions
    X1 = fliterd_test_df1[features]
    X0 = fliterd_test_df0[features]

    # For the output equal to 1 apply MR to evaluate the performance
    y1_pred = xgbR.predict(X1)
    y1_pred = y1_pred.clip(0, 100)
    y1 = fliterd_test_df1["percent_houses_damaged"]

    # For the output equal to 0 apply M1 to evaluate the performance
    y0_pred = xgb.predict(X0)
    y0_pred = y0_pred.clip(0, 100)
    y0 = fliterd_test_df0["percent_houses_damaged"]

    fliterd_test_df0["predicted_percent_damage"] = y0_pred
    fliterd_test_df1["predicted_percent_damage"] = y1_pred

    # Join two dataframes together
    join_test_dfs = pd.concat([fliterd_test_df0, fliterd_test_df1])

    y_join = join_test_dfs["percent_houses_damaged"]
    y_pred_join = join_test_dfs["predicted_percent_damage"]

    pred_df = pd.DataFrame(columns=["y_all", "y_pred_all"])
    pred_df["y_all"] = y_join
    pred_df["y_pred_all"] = y_pred_join

    # bin_index = np.digitize(pred_df["y_all"], bins=binsP2)

    # Join data with y_all and y_all_pred
    df_data_w_pred = pd.merge(pred_df, df_data, left_index=True, right_index=True)
    # Join data with grid_point_id typhoon_year
    df_data_w_pred_grid = pd.merge(
        df[["grid_point_id", "typhoon_year"]],
        df_data_w_pred,
        left_index=True,
        right_index=True,
    )
    df_data_w_pred_grid.sort_values("y_pred_all", ascending=False)

    # join with weights df
    join_df = df_data_w_pred_grid.merge(df_weight, on="grid_point_id", how="left")

    # Indicate where values are valid and not missing
    join_df = join_df.loc[join_df["weight"].notna()]

    # Multiply weight by y_all and y_pred_all
    join_df["weight*y_pred*houses"] = (
        join_df["y_pred_all"] * join_df["weight"] * join_df["total_houses"] / 100
    )
    join_df["weight*y*houses"] = (
        join_df["y_all"] * join_df["weight"] * join_df["total_houses"] / 100
    )
    join_df["weight*houses"] = join_df["weight"] * join_df["total_houses"]

    join_df.sort_values("y_pred_all", ascending=False)

    # Groupby by municipality and typhoon_name with sum as the aggregation function
    agg_df = join_df.groupby(["ADM3_PCODE", "typhoon_name", "typhoon_year"]).agg("sum")

    # Normalize by the sum of the weights
    agg_df["y_pred_norm"] = (
        agg_df["weight*y_pred*houses"] / agg_df["weight*houses"] * 100
    )
    agg_df["y_norm"] = agg_df["weight*y*houses"] / agg_df["weight*houses"] * 100

    # Drop not required column y and y_pred before multiplying by weight
    agg_df.drop("y_all", axis=1, inplace=True)
    agg_df.drop("y_pred_all", axis=1, inplace=True)

    # Remove rows with NaN after normalization
    final_df = agg_df.dropna()
    final_df = final_df.reset_index()
    print(len(final_df))
    # print(final_df)

    # Intersection of two datasets grid and municipality
    # Rename a column
    final_df = final_df.rename(
        columns={"ADM3_PCODE": "Mun_Code", "typhoon_name": "typhoon"}
    )

    # print(final_df)
    # Merge DataFrames based on 'typhoon_name' and 'Mun_Code'
    merged_df = pd.merge(
        final_df, df_mun_merged, on=["Mun_Code", "typhoon"], how="inner"
    )
    print(len(merged_df))
    # print(merged_df)

    # Calculate RMSE & Average Error in total for converted grid_based model to Mun_based
    #if (len(merged_df["y_norm"])) != 0:
    rmse = sqrt(mean_squared_error(merged_df["y_norm"], merged_df["y_pred_norm"]))
    ave = (merged_df["y_pred_norm"] - merged_df["y_norm"]).sum() / len(
        merged_df["y_norm"]
    )

    print(f"RMSE for grid_based model: {rmse:.2f}")
    print(f"Average Error for grid_based model: {ave:.2f}")

    RMSE["all"].append(rmse)
    AVE["all"].append(ave)

    bin_index = np.digitize(merged_df["y_norm"], bins=binsP2)

    #for bin_num in range(1, 6):
    if len(merged_df["y_norm"][bin_index == bin_num]) > 0:

        mse_idx = mean_squared_error(
            merged_df["y_norm"][bin_index == bin_num],
            merged_df["y_pred_norm"][bin_index == bin_num],
        )
        rmse = np.sqrt(mse_idx)

        ave = (
            merged_df["y_pred_norm"][bin_index == bin_num]
            - merged_df["y_norm"][bin_index == bin_num]
        ).sum() / len(merged_df["y_norm"][bin_index == bin_num])

        RMSE[bin_num].append(rmse)
        AVE[bin_num].append(ave)

16260
5002
RMSE for grid_based model: 4.56
Average Error for grid_based model: -0.06
16345
4936
RMSE for grid_based model: 5.13
Average Error for grid_based model: -0.06
16388
5020
RMSE for grid_based model: 4.74
Average Error for grid_based model: -0.14
16445
4986
RMSE for grid_based model: 5.12
Average Error for grid_based model: -0.21
16345
5008
RMSE for grid_based model: 4.78
Average Error for grid_based model: -0.28
16284
4962
RMSE for grid_based model: 4.58
Average Error for grid_based model: -0.07
16179
5013
RMSE for grid_based model: 4.69
Average Error for grid_based model: 0.04
16456
5030
RMSE for grid_based model: 5.10
Average Error for grid_based model: -0.13
16152
4917
RMSE for grid_based model: 4.68
Average Error for grid_based model: 0.01
16400
5083
RMSE for grid_based model: 4.59
Average Error for grid_based model: 0.02
16042
4945
RMSE for grid_based model: 4.48
Average Error for grid_based model: -0.08
16269
4871
RMSE for grid_based model: 4.25
Average Error for grid_ba

In [58]:
# Define a function to plot RMSEs
def rmse_ave_mean(rmse, ave):

    # Mean of RMSE and Standard deviation
    m_rmse = statistics.mean(rmse)
    sd_rmse = statistics.stdev(rmse)

    m_ave = statistics.mean(ave)
    sd_ave = statistics.stdev(ave)

    print(f"mean_RMSE: {m_rmse:.2f}")
    print(f"stdev_RMSE: {sd_rmse:.2f}")

    print(f"mean_average_error: {m_ave:.2f}")
    print(f"stdev_average_error: {sd_ave:.2f}")

In [59]:
print("RMSE and Average Error in total", "\n")
rmse_ave_mean(RMSE["all"], AVE["all"])

RMSE and Average Error in total 

mean_RMSE: 4.73
stdev_RMSE: 0.28
mean_average_error: -0.06
stdev_average_error: 0.09


In [60]:
for bin_num in range(1, 6):

    print(f"\n RMSE and Average Error per bin {bin_num}\n")
    rmse_ave_mean(RMSE[bin_num], AVE[bin_num])


 RMSE and Average Error per bin 1

mean_RMSE: 0.39
stdev_RMSE: 0.20
mean_average_error: 0.04
stdev_average_error: 0.01

 RMSE and Average Error per bin 2

mean_RMSE: 1.94
stdev_RMSE: 0.17
mean_average_error: 0.67
stdev_average_error: 0.05

 RMSE and Average Error per bin 3

mean_RMSE: 5.64
stdev_RMSE: 0.59
mean_average_error: 1.00
stdev_average_error: 0.35

 RMSE and Average Error per bin 4

mean_RMSE: 12.48
stdev_RMSE: 0.90
mean_average_error: -4.53
stdev_average_error: 0.97

 RMSE and Average Error per bin 5

mean_RMSE: 31.67
stdev_RMSE: 3.97
mean_average_error: -25.39
stdev_average_error: 3.97
